# Data Analysis

In this notebook we will try to understand the topics we have in our database by answering the following questions: 
* Which information our data contain?
* If our data have missing values and duplicates?
* How the topics are organized? 
* Which topics appear most? 
* Are all the topics in english?
* How we can split the phrases into phrases?

## Load the data 

Lets load the data.

We download our DB data and now we want to explore the data of the books we have to analyze them. 

Our main goal is to learn about the topics of the books and parse them to phrases that we can work with in the next steps of the projects.

We will extract the following information from our data:
* **001 - book mms id**
* **020 - book isbn**
* **245 - book title**
* **520 - book description**
* **650 - book topics**
* **300 - book number of pages**

In [1]:
"""
Extracting from the MARC file the following informations: 
    001 - book mms id
    020 - book isbn
    245 - book title
    520 - book description
    650 - book topics
    100 - book author
    300 - book number of pages
"""
import xml.etree.ElementTree as ET
import re
xml_tree = ET.parse("BIBLIOGRAPHIC_64846678830003941_1.xml")
xml_root = xml_tree.getroot()
enter = False
# lists for the books informations
ids=[]
titles = []
descriptions = []
records_isbn = []
topic_by_record = []
authors = []
book_pages = []
for record in xml_root:
    # local varaiables to extract the data from file
    description = None
    isbn = None
    title = None
    mms_id = None
    author=None
    record_topics= []
    pages = None
    # check each of the subfield and extract the information
    for field in record.findall('controlfield'):
        if field.get('tag') == '001':
            mms_id = field.text
            break
    for field in record.findall('datafield'):
        if not enter:
            enter = True
        if field.get('tag') == '650':
            for subfield in field.findall('subfield'):
                if subfield.get('code') == 'a':
                    record_topics.append(subfield.text)
        elif field.get('tag') == '245':
            for subfield in field.findall('subfield'):
                if subfield.get('code') == 'a':
                    title = subfield.text
                    if not title[-1].isalpha():
                        title = title[:-1]     
        elif field.get('tag') == '020':
            for subfield in field.findall('subfield'):
                if subfield.get('code') == 'a':
                    isbn = subfield.text
        elif field.get('tag') == '520':
            for subfield in field.findall('subfield'):
                if subfield.get('code') == 'a':       
                    description = subfield.text
        elif field.get('tag') == '100': 
            for subfield in field.findall('subfield'):
                if subfield.get('code') == 'a':
                    author = subfield.text
        elif field.get('tag') == '300': 
            for subfield in field.findall('subfield'):
                if subfield.get('code') == 'a':
                    b_pages = re.sub("[^0-9]", "", subfield.text)
                    if b_pages != "":
                        pages = int(b_pages) 
                    else:
                        pages = None
    topic_by_record.append(record_topics)
    titles.append(title)
    ids.append(mms_id)
    descriptions.append(description)
    records_isbn.append(isbn)
    authors.append(author)
    book_pages.append(pages)

# Data Preprocessing and analysis

## Check that we have all the informations we need

We loaded the data. Lets check the different information we extract and analyze them. Firstable we need to check we have all the required and search for missing values/duplicates.

In [2]:
print("titles number:", len(titles))
print(*titles[0:5],sep="\n",end="\n\n")

print("topics number:", len(topic_by_record))
print(*topic_by_record[0:5],sep="\n",end="\n\n")

print("books ids number:", len(ids))
print(*ids[0:5],sep="\n",end="\n\n")

print("book description number:", len(descriptions))
print(*descriptions[0:5],sep="\n",end="\n\n")

print("Authors number:", len(authors))
print(*authors[0:5],sep="\n",end="\n\n")

print("Books isbn number:", len(records_isbn))
print(*records_isbn[0:5],sep="\n",end="\n\n")

print("Books page number:", len(book_pages))
print(*book_pages[0:5],sep="\n",end="\n\n")

titles number: 15881
The Highlander's stolen bride 
Daisy's back in town 
See Jane score 
Deeper than desire 
Too hot to handle 

topics number: 15881
['Romance fiction', 'Romance fiction']
['Man-woman relationships', 'Man-woman relationships']
['Tabloid newspapers', 'Women sportswriters', 'Hockey players', 'Hockey players', 'Tabloid newspapers', 'Women sportswriters']
['Romance fiction', 'Romance fiction']
['Man-woman relationships', 'Man-woman relationships']

books ids number: 15881
990144992260203941
990144992640203941
990144992760203941
990144992830203941
990144992890203941

book description number: 15881
Despite his plans to marry his childhood friend Megan MacPherson, highland laird Derek Hardwicke is irrestibly drawn to Lady Rosalyn Carmichael, a lovely English girl on the run from her evil stepbrother, who will do anything to seize control of her inheritance.
Former cheerleader Daisy Lee Monroe returns to her hometown of Lovett, Texas, and to her offbeat family, and is reuniti

We have 15881 entries in the data and we checked how our data looks.

## Merge all the information to one dataset

Since we have all the data and the number of all our book information are equal lets merge them together for a better analysis.

We will use Pandas dataset for analyze our data.

In [3]:
import pandas as pd
# overide display basic size to get a better view for our datasets:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 5000)
pd.set_option("max_colwidth", 1000)
pd.set_option('display.max_rows', 5000)

#Convert topics to DataFrame
data = pd.DataFrame({"id":ids,
                     "isbn":records_isbn,
                     "title":titles,
                     "topics":topic_by_record,
                     "description":descriptions,
                     "author":authors,
                     "pages":book_pages})

data['topic length'] = data["topics"].str.len()
display(data.head(5))

,id,isbn,title,topics,description,author,pages,topic length
0,990144992260203941,1451631839,The Highlander's stolen bride,"[Romance fiction, Romance fiction]","Despite his plans to marry his childhood friend Megan MacPherson, highland laird Derek Hardwicke is irrestibly drawn to Lady Rosalyn Carmichael, a lovely English girl on the run from her evil stepbrother, who will do anything to seize control of her inheritance.","George, Melanie.",355.0,2
1,990144992640203941,9780060009250,Daisy's back in town,"[Man-woman relationships, Man-woman relationships]","Former cheerleader Daisy Lee Monroe returns to her hometown of Lovett, Texas, and to her offbeat family, and is reunitied with gorgeous bad boy Jackson Lamott Parrish, the man she had left behind.","Gibson, Rachel",370.0,2
2,990144992760203941,9780060009243,See Jane score,"[Tabloid newspapers, Women sportswriters, Hockey players, Hockey players, Tabloid newspapers, Women sportswriters]","While covering the Seattle Chinooks hockey team, reporter Jane Alcott meets career-focused goalie Luc Martineau, but their relationship is threatened by Jane's other occupation--writing the scandalous ""Honey Pie"" adventures.","Gibson, Rachel",375.0,6
3,990144992830203941,9780312992828,Deeper than desire,"[Romance fiction, Romance fiction]","DESCRIPTION: With her family in dire straits, Olivia Hopkins reluctantly agrees to seek a marriage proposal from the aging Earl of Bristol. But the plan goes awry when Olivia finds an erotic volume in the Earl's library. Catching Olivia with the scandalous volume offers Philip Paxton a superb chance to humiliate the father he despises. Using the book as bait, Philip lures Olivia into an electrifying affair that explodes into unbridled lessons of carnal pleasure ...","Holt, Cheryl,",376.0,2
4,990144992890203941,9780312937973,Too hot to handle,"[Man-woman relationships, Man-woman relationships]","Arriving at the residence of the Earl of Winchester to interview for the position of governess to his two young wards in order to escaped an unwanted marriage, Emily Barnett is stunned to discover that the earl is also in the market for a new mistress.","Holt, Cheryl,",342.0,2


## Check for missing values or id duplications

We want to check if we had a missing values in our dataset and also check if we have ID duplications.

In [4]:
import sidetable
print("Display missing values in the data:")
display(data.drop("topics",axis=1).stb.missing(style=True))


if data['id'].duplicated().sum() != 0:
    print("\n"+"The dataframe have ID duplications.\n")
else:
    print("\n"+"The dataframe don't have ID duplicatations.\n")

Display missing values in the data:


,missing,total,percent
description,"12,196","15,881",76.80%
isbn,"9,118","15,881",57.41%
author,"2,256","15,881",14.21%
pages,84,"15,881",0.53%
id,0,"15,881",0.00%
title,0,"15,881",0.00%
topic length,0,"15,881",0.00%



The dataframe don't have ID duplicatations.



We can notice that for some books we have missing values:
* The description of the books is missing in 76.80% of the data, so most of the time we can't use that information.
* The ISBN(Intenational Standard Book Number) is missing in 57.41% of the data. 
* The author name is missing in 14.21% of the data.
* The number of pages is missing in 84 books(0.53% of the data)

In the other categories(ID and title) we don't have missing values. In the topics we will check later on.

## Check the books we have by the title

In [5]:
titles_count = data["title"].value_counts().reset_index()
titles_count.columns=["title","Number of appearence"]
display(titles_count.head(10))
display(titles_count.tail(10))

,title,Number of appearence
0,The life and adventures of Robinson Crusoe,67
1,Robinson Crusoe,17
2,Life and adventures of Robinson Crusoe,14
3,Robinson Crusoe,12
4,The pilgrim's progress,12
5,The pilgrim's progress from this world to that which is to come,11
6,The Swiss family Robinson,11
7,The Life and adventures of Robinson Crusoe,10
8,Paul and Virginia,10
9,"The life and adventures of Robinson Crusoe of York, mariner",9


,title,Number of appearence
14353,Daughters of memory,1
14354,Beauty's punishment,1
14355,The lover of horses,1
14356,The gospel of wealth in the American novel,1
14357,Tuppence ha'penny is a nickel,1
14358,Spanking the maid,1
14359,Daughters of the twilight,1
14360,The night the gods smiled,1
14361,Smoke detector,1
14362,"Sins of the tongue, or, Truth is everything",1


We can see that titles most of the time appears really few times.

## Check the topics and their appearence

In [6]:
topics_count = data["topics"].value_counts().reset_index()
topics_count.columns=["topics","Number of appearence"]
display(topics_count.head(15))
display(topics_count.tail(15))

# check the precentage of the missing values
print("\n")
print("Percentage of the missing values in the topics:",end=" ")
print(100*(len(data[data["topics"].str.len()==0])/len(data)),end="%\n")

,topics,Number of appearence
0,[],1485
1,"[Children, Conduct of life, Children's stories]",198
2,"[Gay men, Gay men]",165
3,"[Castaways, Shipwrecks, Shipwreck survival]",117
4,[Manners and customs],110
5,"[Christian life, Children's stories]",106
6,"[Lesbians, Lesbians]",99
7,"[Children's stories, Children, Conduct of life]",69
8,"[Romance fiction, Romance fiction]",56
9,[Canadian fiction],55


,topics,Number of appearence
10848,"[Children's stories, Child labor, Temperance]",1
10849,"[Families, Christian life, Curiosity, Suffering, Diseases, Humiliation, Animal welfare, Children, Conduct of life]",1
10850,"[Christian life, Mothers and daughters, Charity, Poverty]",1
10851,[Jackdaw],1
10852,"[Truthfulness and falsehood, Friendship, Girls, Conduct of life]",1
10853,"[Fishing stories, Camping, Trapping]",1
10854,"[Children, Conduct of life, Friendship, Quarreling, Theft, Evidence, Circumstantial, Temperance, Courage, Rabies]",1
10855,"[Boys, Conduct of life, First year teachers, Teacher-student relationships, Students, Private schools, Electric generators, Physical sciences, Courtship, Country life, Brigands and robbers]",1
10856,"[Blindness, Brothers and sisters, Charity, Christian life, Poverty, Success, Artists]",1
10857,"[Fathers and daughters, Poverty, Farm life, Animals, Imaginary places, Adventure stories, Carnivals, Leisure, Water carriers (Persons), Monks, Children's stories]",1




Percentage of the missing values in the topics: 9.350796549335683%


We can get the following conclusions from the tables we displayed:
* The topics can repeat a few times - the topics nedd to be normalized.
* The topics are usually short pharses of 1 or 2 words.
* In 9.3% of the data(almost 10%) the book don't have topic. which is bad - because we wanted to use this subfield to find similartity between books.

## Normalization of topics to phrases

lets try to split the topics to phrases.

In [7]:
normalized_topics = []
for record_topics in topic_by_record:
    rec_topics = list(set(record_topics))
    normalized_topic_record = []
    for topic in rec_topics:
        if '/' in topic:
            topic = topic.replace("/",",")
        elif "--" in topic:
            topic = topic.replace("--",",")
        elif "—" in topic:   
            topic = topic.replace("—",",")
        normalized_topic_record = normalized_topic_record+(topic.split(", "))   
    normalized_topics.append(normalized_topic_record)  
    
for i in range(len(normalized_topics)):
    if "" in normalized_topics[i]:
        normalized_topics[i].remove("")
    for j in range(len(normalized_topics[i])):  
        if len(normalized_topics[i][j])!=0 and normalized_topics[i][j][-1] in ["."," "]:
            normalized_topics[i][j] = normalized_topics[i][j][:-1] 
            
print(*normalized_topics[160:175],sep="\n")    

['Child labor', 'Flour mills', 'Conduct of life', 'Children', 'Jealousy', 'Nannies']
['Christian life', "Children's stories"]
['Christian life', 'Conduct of life', 'Children', 'Brothers and sisters', 'Cheerfulness']
['Slavery', 'African Americans', 'Plantation life', 'Cruelty', 'Fugitive slaves', 'Christian life', 'Slaves']
['Christian life', "Children's stories"]
['Cats', 'Aunts', 'Poor', 'Boys', 'Animal welfare', 'Conduct of life', 'Fairies', 'Cousins', 'Ponies', 'Dreams', 'Birds']
['Horses', 'Aunts', 'Steamboats', 'Voyages and travels', 'Storytelling', 'Conduct of life', 'Children', 'Families']
['Laziness', 'Honesty', 'Conduct of life', 'Children', 'Brothers and sisters', 'Parent and child']
['Laziness', 'Country life', 'Animal welfare', 'Conduct of life', 'Kindness']
['Bees', 'Christian life', 'Conduct of life', 'Kindness', 'Children', 'Diligence', 'Mothers and daughters']
['Magic', 'Elves', 'Kings and rulers', 'Aunts', 'Fairies', 'Conduct of life', 'Children']
['Pride and vanity',

Lets check how much pharses we have in each topics.

In [8]:
multi_phrase_topic= []
one_phrase_topic = []
no_phrase_topics = []
for topic in normalized_topics:
    if len(topic)>1:
        multi_phrase_topic.append(topic) 
    elif len(topic)==1:  
        one_phrase_topic.append(topic)
    else:
        no_phrase_topics.append(topic)
print(len(multi_phrase_topic)) 
print(len(one_phrase_topic))
print(len(no_phrase_topics))

11455
2941
1485


Check the results of the normaliztion

In [9]:
import random

print("Percentage of the topics that have multiple phrases:",end=" ")
print(100*(len(multi_phrase_topic)/len(normalized_topics)),end="%\n")

print("Print example of 40 topics in the data:")
random_row = random.randint(0,len(normalized_topics)-41)
print(*normalized_topics[random_row:random_row+41],sep="\n")            

Percentage of the topics that have multiple phrases: 72.13021850009446%
Print example of 40 topics in the data:
['Show jumping', 'Lesbians', 'Teenage girls']
[]
['Thieves']
['Young women']
[]
['Actors', 'Theater']
['Romance fiction', 'Christian fiction']
[]
['Sadomasochism']
['Books and reading']
['Legal stories', 'Family secrets', 'Lawyers', 'Irish Americans']
['Rabbis']
['Kidnapping']
[]
['Strangers', 'Country life']
['Immigrants', 'Hockey stories', 'Canadian (English)', 'Poor families', 'Jews']
['Narration (Rhetoric)', 'Storytelling', 'Fiction', 'Identity (Psychology) in literature', 'Multiculturalism', 'Politics and literature']
["Children's stories", 'English', 'Fantasy fiction', 'English', 'Characters and characteristics']
['HIV-positive persons']
['Canadian fiction (English)', 'Short stories', 'Canadian (English)', 'Comic books', 'strips', 'etc', 'Canadian (English)']
[]
['Private investigators']
['Rural families', 'Rejection (Psychology)', 'Foundlings', 'Triangles (Interpersona

Check if all the topics are in english. We will use WordNet and the enchant library to check that. Lets define a topic that is not english as a topic that all the words in it are not recognized by our tools.

In [10]:
from nltk.corpus import wordnet as wn
import enchant
import re
us_d = enchant.Dict("en_US")
gb_d = enchant.Dict("en_GB")

english_topics = []
not_english_topics = []
for topic in normalized_topics:
    if len(topic)==0:
        continue
    words = 0
    words_in_other_language = 0
    for phrase in topic:
        word_finder = re.split(r"[^a-zA-Z]",phrase)
        words+=len(word_finder)
        for word in word_finder:
            if len(word)==0:
                words -= 1
            elif not (us_d.check(word) or gb_d.check(word)) and len(wn.synsets(word))==0:
                words_in_other_language += 1
                
    if words<=3 and words_in_other_language*2<words:
        english_topics.append(topic)
    elif words_in_other_language*2.5<words:     
        english_topics.append(topic)
    else:
        not_english_topics.append(topic)
print("The number of books with topics that are not in english:",len(not_english_topics))

The number of books with topics that are not in english: 122


In [11]:
print(*not_english_topics[55:65],sep="\n",end="\n\n")

['Birthparents']
['Women', 'Anthologie', 'Erotische Erzählung', 'American fiction', 'Erotic stories', 'American', 'Erotic stories', 'American', 'Frauenerzählung']
['Ayahuasca']
['Japanese Americans', "Américains d'origine japonaise"]
['Roman canadien-anglais', 'Nouvelles canadiennes-anglaises', 'Canadian fiction', 'Short stories', 'Canadian']
['Enfants', 'Littérature de jeunesse canadienne-anglaise', 'Meilleurs ouvrages', 'Best books', "Children's literature", 'Canadian (English)', 'Children', 'Roman historique canadien-anglais', 'Historical fiction', 'Canadian (English)']
['Écrits de femmes antillais (anglais)', 'Lesbians', 'Lesbianisme', "Lesbians' writings", 'Nouvelles antillaises (anglaises)', 'Short stories', 'Caribbean (English)', 'Lesbiennes', 'Lesbianism']
['Sadomasochisme', 'Sadomasochism', 'Prostituées', 'Prostitutes']
['Verteltheorie', 'Narration (Rhetoric)', 'Roman', 'Fiction', 'Prosa', 'Narration', 'Analyse du discours narratif', 'English fiction', 'Erzähltechnik']
['Narra

### Create a dataframe with the final information

In [12]:
book_dataset = pd.DataFrame({"id":ids,
                             "isbn":records_isbn,
                             "title":titles,
                             "topics":normalized_topics,
                             "description":descriptions,
                             "author":authors,
                             "pages":book_pages})
display(book_dataset.head(5))

,id,isbn,title,topics,description,author,pages
0,990144992260203941,1451631839,The Highlander's stolen bride,[Romance fiction],"Despite his plans to marry his childhood friend Megan MacPherson, highland laird Derek Hardwicke is irrestibly drawn to Lady Rosalyn Carmichael, a lovely English girl on the run from her evil stepbrother, who will do anything to seize control of her inheritance.","George, Melanie.",355.0
1,990144992640203941,9780060009250,Daisy's back in town,[Man-woman relationships],"Former cheerleader Daisy Lee Monroe returns to her hometown of Lovett, Texas, and to her offbeat family, and is reunitied with gorgeous bad boy Jackson Lamott Parrish, the man she had left behind.","Gibson, Rachel",370.0
2,990144992760203941,9780060009243,See Jane score,"[Hockey players, Tabloid newspapers, Women sportswriters]","While covering the Seattle Chinooks hockey team, reporter Jane Alcott meets career-focused goalie Luc Martineau, but their relationship is threatened by Jane's other occupation--writing the scandalous ""Honey Pie"" adventures.","Gibson, Rachel",375.0
3,990144992830203941,9780312992828,Deeper than desire,[Romance fiction],"DESCRIPTION: With her family in dire straits, Olivia Hopkins reluctantly agrees to seek a marriage proposal from the aging Earl of Bristol. But the plan goes awry when Olivia finds an erotic volume in the Earl's library. Catching Olivia with the scandalous volume offers Philip Paxton a superb chance to humiliate the father he despises. Using the book as bait, Philip lures Olivia into an electrifying affair that explodes into unbridled lessons of carnal pleasure ...","Holt, Cheryl,",376.0
4,990144992890203941,9780312937973,Too hot to handle,[Man-woman relationships],"Arriving at the residence of the Earl of Winchester to interview for the position of governess to his two young wards in order to escaped an unwanted marriage, Emily Barnett is stunned to discover that the earl is also in the market for a new mistress.","Holt, Cheryl,",342.0


In [13]:
import spacy
import random
from statistics import mean
nlp = spacy.load("en_core_web_lg")

print("Choose 2 random books with topics")
book_1 = 0
book_2 = 0
while True:
    book_1 = random.randint(0,len(book_dataset))
    book_2 = random.randint(0,len(book_dataset))
    if len(book_dataset.iloc[book_1]['topics'])!=0 and len(book_dataset.iloc[book_2]['topics'])!=0:
        break

print("Compare between those 2 books:\n")
print("Title:",book_dataset.iloc[book_1]['title'],"\nTopics:",book_dataset.iloc[book_1]['topics'],end="\n\n")
print("Title:",book_dataset.iloc[book_2]['title'],"\nTopics:",book_dataset.iloc[book_2]['topics'],end="\n\n")
    
similarity_results = []
for phrase1 in book_dataset.iloc[book_1]["topics"]:
    doc1 = nlp(phrase1)
    top_similarity = 0
    for phrase2 in book_dataset.iloc[book_2]["topics"]:
        doc2 = nlp(phrase2) 
        res = doc1.similarity(doc2)
        if top_similarity < res:
            top_similarity = res
            if top_similarity==1:
                break
    similarity_results.append(top_similarity)    

print("Results:",similarity_results)
print("The mean similarity between the 2 books is:",mean(similarity_results))

Choose 2 random books with topics
Compare between those 2 books:

Title: A lesson in love  
Topics: ['Lesbian students', 'Young women', 'Lesbian youth', 'Coming-of-age stories', 'Orphans']

Title: Stony glances  
Topics: ['Gay men']

Results: [0.6247083550590737, 0.7784994118412452, 0.7299363917192144, 0.4627758412536999, 0.23531509790601265]
The mean similarity between the 2 books is: 0.5662470195558492
